### References

<ul>
    <li>Trending twitter<br></li>
    https://twitter-trends.iamrohit.in/
    <li>Removing accented characters<br></li>
    https://stackoverflow.com/a/2633310/13557629
    <li>Importing package from filepath</li>
    https://stackoverflow.com/a/50395128/13557629
</ul>

In [2]:
import json
import yaml
import re
import logging
import numpy as np 
import pandas as pd
import connection as tc

from datetime import datetime
from time import sleep
from unidecode import unidecode

In [3]:
import importlib
import sys

spec_src = importlib.util.spec_from_file_location(
    'src', 
    '../__init__.py')
m = importlib.util.module_from_spec(spec_src)
sys.modules[spec_src.name] = m
spec_src.loader.exec_module(m)

from src import utils

In [3]:
importlib.reload(connection)

NameError: name 'connection' is not defined

In [4]:
logger = utils.get_logger('extraction', datetime.now(), 'Testing log creation')

INFO:root:Opened config file at: /home/ruskirin/Documents/Projects/lin-que-dropping/config/general_config.yml


In [5]:
gen_conf = utils.get_config()
conf = utils.get_config('e')

### Extraction

In [6]:
es_conn = tc.TwitterConnection(
    lang='es',
    bearer_path = 'academic/token',
    is_archive=True)

# pt_conn = connection.TwitterConnection() 

In [7]:
conjugations = pd.read_excel(utils.get_project_root()/gen_conf['file_paths']['verb_conjug'])
conjugations.head(3)

,verb_type,verb,indicativo,imperativo,subjuntivo,gerundio,gerundio_compuesto,infinitivo,infinitivo_compuesto,participio_pasado
0,Stative,ver,veía visto verías vi vimos verían ves v...,vean ve vea veamos ved,veáis visto vieras vieren viesen veas vi...,viendo,visto,ver,visto,visto
1,Stative,jurar,jurarán juramos jurarías jurabas juraría ...,jurad jura juren jure juremos,jurare jurareis jurase jurara juraren jur...,jurando,jurado,jurar,jurado,jurado
2,Stative,decir,decíais dijeron decimos dice dirían decía...,di digan digamos decid diga,dijere dijéremos dijesen dijeses dicho di...,diciendo,dicho,decir,dicho,dicho


In [8]:
topics = conjugations.loc[:, ['verb', 'indicativo']]
topics['indicativo'] = topics['indicativo'].str.replace(r'\s+', ' OR ')\
                                           .apply(lambda x: '(' + x + ')')
topics = topics.to_dict('records')

In [9]:
time = utils.get_str_datetime_now()

In [10]:
# List of verbs that shouldn't be pulled
finished = conf['filled_verbs']

In [13]:
finished

['decir',
 'ver',
 'querer',
 'saber',
 'creer',
 'esperar',
 'pensar',
 'jurar',
 'confesar',
 'prometer',
 'mostrar',
 'demostrar']

In [12]:
save_path = utils.make_dir(utils.get_save_path('e', 'twitter', False, 'es')/time)

for i, topic in enumerate(topics):
    # Skip verbs that were already pulled or are not desired
    if topic['verb'] in finished:
        continue
    
    pages = es_conn.paginate(save_path, 
                             (topic['verb'], topic['indicativo']), 
                             batch_size=100,
                             batch_num=5,
                             sleep_sec=1)
    
    print(f'Retrieved total {pages} pages for {topic["verb"]}.')
    
    finished.append(topic['verb'])

Retrieved total 5 pages for jurar.
Retrieved total 5 pages for confesar.
Retrieved total 5 pages for prometer.
Retrieved total 5 pages for mostrar.
Retrieved total 5 pages for demostrar.


KeyboardInterrupt: 